In [1]:
import pandas as pd

In [2]:

from urllib.request import urlopen
import requests 
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select


import pandas as pd

driver_path = '/Users/zacharywong/Downloads/chromedriver'
service = Service(driver_path)

url = 'https://www.shazam.com/charts/top-200/united-states'

driver = webdriver.Chrome(service = service)
driver.get(url)
xpath = '//select[@data-shz-type="city"]'
#id = 'charts.ue-download-csv'

element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, xpath)))
try:
    cities = driver.find_element(By.XPATH, xpath)
    dropdown = Select(cities)
    
except:
    cities = driver.find_element(By.XPATH, xpath)
    dropdown = Select(cities)




In [14]:
#cities
citylist = []

for city in dropdown.options:
    citynormalized = city.text.replace(',', '').replace(' ', '-')
    citylist.append(citynormalized)
citylist.pop(0)
len(citylist)
    

    

202

In [4]:
#get csv links for each city in dictionary 

cityURL = {}
baseurl = 'https://www.shazam.com/services/charts/csv/top-50/united-states/'
for city in citylist:
    csvurl = baseurl + str(city)
    cityURL[city] = csvurl

cityURL
    
#df = pd.read_csv('https://www.shazam.com/services/charts/csv/top-50/united-states/albany', skiprows = 2)

    

{'Aberdeen-SD': 'https://www.shazam.com/services/charts/csv/top-50/united-states/Aberdeen-SD',
 'Aberdeen-WA': 'https://www.shazam.com/services/charts/csv/top-50/united-states/Aberdeen-WA',
 'Abilene': 'https://www.shazam.com/services/charts/csv/top-50/united-states/Abilene',
 'Akron': 'https://www.shazam.com/services/charts/csv/top-50/united-states/Akron',
 'Albany': 'https://www.shazam.com/services/charts/csv/top-50/united-states/Albany',
 'Albuquerque': 'https://www.shazam.com/services/charts/csv/top-50/united-states/Albuquerque',
 'Allentown': 'https://www.shazam.com/services/charts/csv/top-50/united-states/Allentown',
 'Anaheim': 'https://www.shazam.com/services/charts/csv/top-50/united-states/Anaheim',
 'Anchorage': 'https://www.shazam.com/services/charts/csv/top-50/united-states/Anchorage',
 'Asheville': 'https://www.shazam.com/services/charts/csv/top-50/united-states/Asheville',
 'Atlanta': 'https://www.shazam.com/services/charts/csv/top-50/united-states/Atlanta',
 'Aurora': 'h

In [5]:
#Create Panda Dataframe
columnsPart = ["Song Name", "Artist"]
columns = columnsPart + citylist
dfCityTop20 = pd.DataFrame(columns = columns)
dfCityTop20

,Song Name,Artist,Aberdeen-SD,Aberdeen-WA,Abilene,Akron,Albany,Albuquerque,Allentown,Anaheim,...,Virginia-Beach,Warren-OH,Washington-D.C.,Wichita,Wilmington,Woonsocket,Worcester,Yakima,Youngstown,Yuma


In [6]:
# A Helper Function to fill in Song Rankings in the dfCityTop20 dataframe 

def updateSongRank(dfCity, SongIndex, dfCityTop20, SongLimit, CityName):
    
    #Append top 20 songs to the CityTop20 Dataframe 
    SongRank = 0
    while SongRank < SongLimit: 
        SongName =  dfCity.at[SongRank, 'Title']
        SongArtist =  dfCity.at[SongRank, 'Artist']
        
        # append new song/artist to dataframe if not already inside 
        if SongName not in dfCityTop20['Song Name'].unique():
            dfCityTop20 = dfCityTop20.append({"Song Name": SongName, "Artist": SongArtist}, ignore_index = True)
            global SongIndexCounter
            SongIndex.update({SongName: SongIndexCounter})  
            SongIndexCounter += 1
            #print(SongName)
            
        # fill in the value (rank) for the song for the specific city if not the same as previous day
        # update rank for next song
        if (dfCityTop20.at[SongIndex.get(SongName), CityName] != SongRank+1):
            dfCityTop20.at[SongIndex.get(SongName), CityName] = SongRank+1
        SongRank += 1
        
    #print(SongIndex)
    #Reset song rank
    return dfCityTop20

In [7]:

# Keeps track of the song and its index in the dfCityTop20 dataframe 
SongIndex = {}
SongIndexCounter = 0
SongLimit = 20


# Loop through each city 
for cityName in citylist: 
    cityurl = cityURL.get(cityName)
    dfCity = pd.read_csv(cityurl, skiprows = 2)
    dfCityTop20 = updateSongRank(dfCity, SongIndex, dfCityTop20, SongLimit, cityName)
    
dfCityTop20

,Song Name,Artist,Aberdeen-SD,Aberdeen-WA,Abilene,Akron,Albany,Albuquerque,Allentown,Anaheim,...,Virginia-Beach,Warren-OH,Washington-D.C.,Wichita,Wilmington,Woonsocket,Worcester,Yakima,Youngstown,Yuma
0,Big Energy,Latto,1,1,4,3,5,2,2,1,...,3,4,10,5,3,NaN,6,6,12,2
1,Heat Waves,Glass Animals,2,NaN,3,10,NaN,3,NaN,6,...,NaN,10,14,3,11,NaN,14,5,NaN,3
2,Cold Heart (PNAU Remix),Elton John & Dua Lipa,3,NaN,1,1,1,1,1,2,...,5,13,4,1,2,NaN,1,1,1,1
3,I Should Live In Salt,The National,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Right Down The Line,Gerry Rafferty,5,NaN,8,NaN,8,12,17,4,...,NaN,NaN,3,NaN,NaN,NaN,5,7,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,Iron Fist Main Titles,Trevor Morris,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,16,NaN,NaN,NaN,NaN
453,Holy Diver,Dio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,18,NaN,NaN,NaN,NaN
454,Try,Sir Juan Mutant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,19,NaN,NaN,NaN,NaN
455,Main Titles,Marco Beltrami,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,20,NaN,NaN,NaN,NaN


'Right Down The Line'

In [13]:
dfCityTop20.to_csv('/Users/zacharywong/github/zacharywong2023/AtlanticRecords/IntermediateDataFrames/50CityDataTest.csv', index=False)